In [3]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import thinc.extra.datasets
import pandas as pd
import spacy
from spacy.util import minibatch, compounding
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from numpy import array, unique, array_equal
max_fatures = 1000


####
#Windows instructions
#
# open anaconda terminal as admin
# type in (or copy paste)
#
#    conda create -n py36 python=3.6 anaconda

#   Hit y, then enter, etc
## DO NOT STOP THIS PROCESS


# then type

# activate py36

# then type

# conda install -c conda-forge spacy
# conda install -c conda-forge tensorflow 
# conda install -c conda-forge keras 

# hitting [y] key and enter as required
def load_data():
    print("Loading Data...")
    PrimaryEmotion = pd.read_csv('emotion.xls.csv')
    print("Data Loaded")
    return (PrimaryEmotion['sentence'], array(PrimaryEmotion['emotion']))



def sort_to_2_emotions(sentence_list, emotion_list):
    sorted_list = []
    sorted_emo = []
    for (data, emo) in zip(sentence_list, emotion_list):
        if (emo == 'Joy' or emo == 'Love' or emo == 'Optimism' or emo == 'Awe' or emo == 'Trust' or emo == 'Joy'):
            sorted_list.append(data)
            sorted_emo.append("positive")
        if (emo == 'anger' or emo == 'Disgust' or emo == 'Sadness' or emo == 'Aggression' or emo == 'Contempt' or emo == 'Disapproval' or emo == 'Remorse' or emo == 'Fear' or emo == 'Anger' ):
            sorted_list.append(data)
            sorted_emo.append("negative")
    return (sorted_list, sorted_emo)



def tokenize(sentences):
    print("Tokenizing...")
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(sentences)
    X = tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(X)
    print("Tokenization Completed")
    return X



def train(tokened_sentences, emotion_list):
    print("Training...")
    embed_dim = 128
    lstm_out = 196
    test_percent=.20

    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = tokened_sentences.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    
    # len(set(emotion_list)) is a hacky way of geting the number of unique elements
    # in a regualar python list (non-numpy)
    model.add(Dense(unique(emotion_list).size,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    #print(model.summary())

    Y = pd.get_dummies(emotion_list).values
    X_train, X_test, Y_train, Y_test = train_test_split(tokened_sentences,Y, test_size = test_percent, random_state = 42)
    #print(X_train.shape,Y_train.shape)
    #print(X_test.shape,Y_test.shape)

    batch_size = 32
    model.fit(X_train, Y_train, epochs = 100, batch_size=batch_size, verbose = 1)
    print("Training Completed")
    print("Testing Against Control... (% of the data) ", test_percent)
    score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
    print("Score   :", score)
    print("Accuracy:", acc)
    return (model, X_test)

### When Splitting data (train_test_split), we don't retain where in the 
### origional set the data is located, thus it takes
### a little trickery to see the results while while only testing against untrained data

def test(model, X_test, tokenized_data, sentence_list, emotion_list):
    predictions = model.predict(tokenized_data, batch_size=32)
    error = 0
    for i in range(len(predictions)):
     
        #  "If the current tokenized data array is in X_test (untrained tokenized arrays)
        
        if(any(array_equal(tokenized_data[i], x) for x in X_test)):
            pos = list(predictions[i]).index(max(predictions[i]))
            print("\n\n")
            print(sentence_list[i])
            print("\nPredicted emotion: ", unique(emotion_list)[pos])
            print("Actual emotion   : ", emotion_list[i])
            if (unique(emotion_list)[pos] != emotion_list[i]):
                error += 1
    print("%  Accuracy when against untrained set: ", 1- (float(error) / len(X_test)))

    
    
#################################
#############MAIN################
#################################
#   jupyter notebook is weird   #




(sentence_list, emotion_list) = load_data()


### Comment the Below line for all 18 emotions. This sorts into "positive" and "negative"
sentence_list, emotion_list = sort_to_2_emotions(sentence_list, emotion_list)

tokenized_data = tokenize(sentence_list)

(model, X_test) = train(tokenized_data, emotion_list)

test(model, X_test, tokenized_data, sentence_list, emotion_list)


Loading Data...
Data Loaded
Tokenizing...
Tokenization Completed
Training...
Epoch 1/100
1227/1227 [==============================] - ETA: 2:18 - loss: 0.6925 - acc: 0.437 - ETA: 1:17 - loss: 0.6933 - acc: 0.500 - ETA: 55s - loss: 0.6926 - acc: 0.562 - ETA: 44s - loss: 0.6918 - acc: 0.56 - ETA: 37s - loss: 0.6895 - acc: 0.58 - ETA: 32s - loss: 0.6879 - acc: 0.58 - ETA: 28s - loss: 0.6843 - acc: 0.61 - ETA: 25s - loss: 0.6876 - acc: 0.58 - ETA: 23s - loss: 0.6829 - acc: 0.60 - ETA: 21s - loss: 0.6870 - acc: 0.58 - ETA: 19s - loss: 0.6850 - acc: 0.59 - ETA: 17s - loss: 0.6832 - acc: 0.59 - ETA: 16s - loss: 0.6848 - acc: 0.58 - ETA: 15s - loss: 0.6849 - acc: 0.58 - ETA: 14s - loss: 0.6858 - acc: 0.58 - ETA: 13s - loss: 0.6831 - acc: 0.58 - ETA: 12s - loss: 0.6824 - acc: 0.59 - ETA: 11s - loss: 0.6830 - acc: 0.58 - ETA: 10s - loss: 0.6832 - acc: 0.58 - ETA: 9s - loss: 0.6815 - acc: 0.5891 - ETA: 9s - loss: 0.6784 - acc: 0.596 - ETA: 8s - loss: 0.6787 - acc: 0.596 - ETA: 7s - loss: 0.6772 -

1227/1227 [==============================] - ETA: 12s - loss: 0.3671 - acc: 0.84 - ETA: 12s - loss: 0.3181 - acc: 0.82 - ETA: 12s - loss: 0.2861 - acc: 0.87 - ETA: 12s - loss: 0.2800 - acc: 0.89 - ETA: 12s - loss: 0.2675 - acc: 0.89 - ETA: 12s - loss: 0.2603 - acc: 0.89 - ETA: 11s - loss: 0.2491 - acc: 0.90 - ETA: 11s - loss: 0.2383 - acc: 0.91 - ETA: 11s - loss: 0.2251 - acc: 0.92 - ETA: 11s - loss: 0.2150 - acc: 0.92 - ETA: 11s - loss: 0.2181 - acc: 0.92 - ETA: 11s - loss: 0.2264 - acc: 0.91 - ETA: 11s - loss: 0.2306 - acc: 0.91 - ETA: 10s - loss: 0.2267 - acc: 0.91 - ETA: 10s - loss: 0.2222 - acc: 0.92 - ETA: 10s - loss: 0.2183 - acc: 0.92 - ETA: 9s - loss: 0.2110 - acc: 0.9228 - ETA: 9s - loss: 0.2155 - acc: 0.918 - ETA: 8s - loss: 0.2112 - acc: 0.921 - ETA: 8s - loss: 0.2101 - acc: 0.921 - ETA: 7s - loss: 0.2176 - acc: 0.919 - ETA: 7s - loss: 0.2184 - acc: 0.921 - ETA: 6s - loss: 0.2189 - acc: 0.921 - ETA: 6s - loss: 0.2210 - acc: 0.921 - ETA: 5s - loss: 0.2212 - acc: 0.922 - ETA:

1227/1227 [==============================] - ETA: 14s - loss: 0.0515 - acc: 1.00 - ETA: 14s - loss: 0.0705 - acc: 0.98 - ETA: 14s - loss: 0.0685 - acc: 0.97 - ETA: 13s - loss: 0.0847 - acc: 0.96 - ETA: 13s - loss: 0.0822 - acc: 0.96 - ETA: 13s - loss: 0.0825 - acc: 0.97 - ETA: 12s - loss: 0.0888 - acc: 0.96 - ETA: 12s - loss: 0.0942 - acc: 0.96 - ETA: 11s - loss: 0.0918 - acc: 0.96 - ETA: 11s - loss: 0.0986 - acc: 0.96 - ETA: 10s - loss: 0.1032 - acc: 0.96 - ETA: 10s - loss: 0.0983 - acc: 0.97 - ETA: 9s - loss: 0.0942 - acc: 0.9736 - ETA: 9s - loss: 0.0916 - acc: 0.975 - ETA: 8s - loss: 0.0935 - acc: 0.972 - ETA: 8s - loss: 0.0894 - acc: 0.974 - ETA: 8s - loss: 0.0868 - acc: 0.976 - ETA: 7s - loss: 0.0879 - acc: 0.975 - ETA: 7s - loss: 0.0848 - acc: 0.977 - ETA: 7s - loss: 0.0842 - acc: 0.976 - ETA: 6s - loss: 0.0827 - acc: 0.977 - ETA: 6s - loss: 0.0838 - acc: 0.977 - ETA: 6s - loss: 0.0852 - acc: 0.974 - ETA: 5s - loss: 0.0839 - acc: 0.975 - ETA: 5s - loss: 0.0830 - acc: 0.975 - ETA:

1227/1227 [==============================] - ETA: 12s - loss: 0.0731 - acc: 0.93 - ETA: 12s - loss: 0.0750 - acc: 0.93 - ETA: 12s - loss: 0.0567 - acc: 0.95 - ETA: 12s - loss: 0.0499 - acc: 0.96 - ETA: 12s - loss: 0.0662 - acc: 0.96 - ETA: 12s - loss: 0.0557 - acc: 0.96 - ETA: 11s - loss: 0.0572 - acc: 0.96 - ETA: 11s - loss: 0.0530 - acc: 0.97 - ETA: 11s - loss: 0.0534 - acc: 0.97 - ETA: 11s - loss: 0.0539 - acc: 0.97 - ETA: 11s - loss: 0.0553 - acc: 0.97 - ETA: 10s - loss: 0.0521 - acc: 0.97 - ETA: 10s - loss: 0.0497 - acc: 0.97 - ETA: 10s - loss: 0.0482 - acc: 0.97 - ETA: 9s - loss: 0.0513 - acc: 0.9792 - ETA: 9s - loss: 0.0497 - acc: 0.980 - ETA: 8s - loss: 0.0507 - acc: 0.979 - ETA: 8s - loss: 0.0495 - acc: 0.980 - ETA: 8s - loss: 0.0501 - acc: 0.980 - ETA: 7s - loss: 0.0485 - acc: 0.981 - ETA: 7s - loss: 0.0473 - acc: 0.982 - ETA: 6s - loss: 0.0477 - acc: 0.983 - ETA: 6s - loss: 0.0481 - acc: 0.982 - ETA: 6s - loss: 0.0511 - acc: 0.981 - ETA: 5s - loss: 0.0524 - acc: 0.981 - ETA:

1227/1227 [==============================] - ETA: 12s - loss: 0.0116 - acc: 1.00 - ETA: 12s - loss: 0.0129 - acc: 1.00 - ETA: 12s - loss: 0.0091 - acc: 1.00 - ETA: 12s - loss: 0.0175 - acc: 0.99 - ETA: 12s - loss: 0.0260 - acc: 0.98 - ETA: 11s - loss: 0.0234 - acc: 0.98 - ETA: 11s - loss: 0.0210 - acc: 0.99 - ETA: 11s - loss: 0.0197 - acc: 0.99 - ETA: 11s - loss: 0.0188 - acc: 0.99 - ETA: 10s - loss: 0.0199 - acc: 0.99 - ETA: 10s - loss: 0.0194 - acc: 0.99 - ETA: 10s - loss: 0.0218 - acc: 0.99 - ETA: 9s - loss: 0.0203 - acc: 0.9928 - ETA: 9s - loss: 0.0191 - acc: 0.993 - ETA: 9s - loss: 0.0210 - acc: 0.993 - ETA: 8s - loss: 0.0211 - acc: 0.994 - ETA: 8s - loss: 0.0199 - acc: 0.994 - ETA: 7s - loss: 0.0190 - acc: 0.994 - ETA: 7s - loss: 0.0192 - acc: 0.993 - ETA: 7s - loss: 0.0182 - acc: 0.993 - ETA: 6s - loss: 0.0179 - acc: 0.994 - ETA: 6s - loss: 0.0173 - acc: 0.994 - ETA: 5s - loss: 0.0214 - acc: 0.993 - ETA: 5s - loss: 0.0207 - acc: 0.993 - ETA: 5s - loss: 0.0219 - acc: 0.992 - ETA:

...

Predicted emotion:  positive
Actual emotion   :  positive



I was absolutely in my element, how did I come over, Jannie?

Predicted emotion:  negative
Actual emotion   :  negative



You want to have a good time and paint the town red this weekend.

Predicted emotion:  negative
Actual emotion   :  positive



The party is over.

Predicted emotion:  positive
Actual emotion   :  negative



Mm, well it's supposed, they're designed to do that aren't they, so at the end of it you're ready to throttle them, when you got the punch line it's on telly.

Predicted emotion:  negative
Actual emotion   :  negative



With education, the world is your oyster.

Predicted emotion:  positive
Actual emotion   :  positive



My heart sinks, the great indefiniteness of the world is spreading through the files like damp.

Predicted emotion:  positive
Actual emotion   :  negative



He's as pleased as Punch about it.

Predicted emotion:  positive
Actual emotion   :  positive



He stares at you in rapture

It's the hymns that are the great stumbling block.

Predicted emotion:  positive
Actual emotion   :  negative



The temptation to take the easy way out was well-nigh irresistible with youngsters who had failed to make their way.

Predicted emotion:  negative
Actual emotion   :  negative



I'm the tar baby.

Predicted emotion:  positive
Actual emotion   :  negative



Lotus was slightly out and the 80 was an unmitigated disaster.

Predicted emotion:  negative
Actual emotion   :  negative



Look before you leap is a maxim impulsive people fail to follow.

Predicted emotion:  negative
Actual emotion   :  negative



Better sleep on it.

Predicted emotion:  negative
Actual emotion   :  negative



As he sees it, the crest of the wave is coming right his way.

Predicted emotion:  negative
Actual emotion   :  positive



It could also provide a dead cert for some future Gold Cup.

Predicted emotion:  negative
Actual emotion   :  positive



Recruiting outsiders is a dead duck for the mome

Actual emotion   :  negative



Rosie was very insecure and a bundle of nerves.

Predicted emotion:  negative
Actual emotion   :  negative



Honestly, you would soon be bored to tears with it.

Predicted emotion:  negative
Actual emotion   :  negative



You, my san, have got a chip on your shoulder.

Predicted emotion:  negative
Actual emotion   :  negative



It would have been rather too close to home.

Predicted emotion:  negative
Actual emotion   :  negative



I don't usually go to pieces like this.

Predicted emotion:  negative
Actual emotion   :  negative



In effect, the message being sent to John and the other hostages was that they had to keep a stiff upper lip and hope that, one day, the kidnappers might let them go.

Predicted emotion:  positive
Actual emotion   :  negative



And of course we're pleased as Punch to put him in print again.

Predicted emotion:  positive
Actual emotion   :  positive



These were wise words from a wise old bird, but Matilda was too steamed

Actual emotion   :  negative



It seems that we've opened a real can of worms with these Dixon complaints.

Predicted emotion:  positive
Actual emotion   :  negative



The Bill will barely scratch the surface of either problem.

Predicted emotion:  positive
Actual emotion   :  negative



For Africa, meanwhile, the 1980s have been an almost unmitigated disaster.

Predicted emotion:  negative
Actual emotion   :  negative



We ran towards our target -- King Kong's massive back -- without a hitch and I reached it just ahead of my friend.

Predicted emotion:  negative
Actual emotion   :  positive



Candidates who come to grief in exams often lack practice in structuring their thoughts into an effective exam answer.

Predicted emotion:  negative
Actual emotion   :  negative



It is an ideal place for a honeymoon but life afterwards must seem all downhill, at least in terms of location.

Predicted emotion:  positive
Actual emotion   :  negative



The Honey Thieves are a band who would 

Actual emotion   :  positive



They'll come to blows with the Kurds, who'll want their own state.

Predicted emotion:  negative
Actual emotion   :  negative



As food can become the ammunition in a battle of wills between children and their parents, so, too, can the process of toilet training.

Predicted emotion:  negative
Actual emotion   :  negative



A bit hot under the collar.

Predicted emotion:  negative
Actual emotion   :  negative



But, deep down, every Russian knows that our strength is in our unity.

Predicted emotion:  negative
Actual emotion   :  positive



He was a strong, healthy lad and as pleased as punch to be working with Dad.

Predicted emotion:  positive
Actual emotion   :  positive



For the remainder of the carol I was on tenterhooks, not daring to take my eyes from the lectern.

Predicted emotion:  negative
Actual emotion   :  negative



Eva was over the moon.

Predicted emotion:  positive
Actual emotion   :  positive



Superintendent Huddleston noted wi